<a href="https://colab.research.google.com/github/CalculatedContent/ww-phys_theory/blob/master/AutoGrad_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## run AutoGrad on TPU




In [2]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4264  100  4264    0     0  26484      0 --:--:-- --:--:-- --:--:-- 26484
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200515 ...
Uninstalling torch-1.5.0+cu101:
Done updating TPU runtime: <Response [200]>
  Successfully uninstalled torch-1.5.0+cu101
Uninstalling torchvision-0.6.0+cu101:
  Successfully uninstalled torchvision-0.6.0+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200515-cp36-cp36m-linux_x86_64.whl...
\ [1 files][ 91.0 MiB/ 91.0 MiB]                                                
Operation completed over 1 objects/91.0 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200515-cp36-cp36m-linux_x86_64.whl...
/ [1 files][119.5 MiB/119.5 MiB]                                                
Operation completed over 1 objec

In [3]:
# imports pytorch
import torch
# imports the torch_xla package
import torch_xla
import torch_xla.core.xla_model as xm
device = xm.xla_device()
print(device)

xla:1


In [4]:
!pip install ipython-autotime
%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=37a02ce4ca440e7d2059194cc34ca3d17d1c67329160c528a9d7ac9c9d4f0d02
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [5]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

import math
import copy

import gc


[('__call__', <function LevelMapper.__call__ at 0x7f3bf294bea0>), ('__init__', <function LevelMapper.__init__ at 0x7f3bf294be18>)]
[('__call__', <function BalancedPositiveNegativeSampler.__call__ at 0x7f3bf28f9950>), ('__init__', <function BalancedPositiveNegativeSampler.__init__ at 0x7f3bf28f98c8>)]
[('__init__', <function BoxCoder.__init__ at 0x7f3bf2890d90>), ('decode', <function BoxCoder.decode at 0x7f3bf2890f28>), ('decode_single', <function BoxCoder.decode_single at 0x7f3bf2893048>), ('encode', <function BoxCoder.encode at 0x7f3bf2890e18>), ('encode_single', <function BoxCoder.encode_single at 0x7f3bf2890ea0>)]
[('__call__', <function Matcher.__call__ at 0x7f3bf2890d08>), ('__init__', <function Matcher.__init__ at 0x7f3bf2890c80>), ('set_low_quality_matches_', <function Matcher.set_low_quality_matches_ at 0x7f3bf2890b70>)]
[('__init__', <function ImageList.__init__ at 0x7f3bf28932f0>), ('to', <function ImageList.to at 0x7f3bf2893268>)]
[('__init__', <function Timebase.__init__ at

In [6]:
import torch
from torch.autograd.gradcheck import zero_gradients
import numpy as np
import math

def batch_diagJ(inputs, output):
  """Computes the diagonal Jacobian by input batches.
  Input: Input for the function for which the Jacobian will
  computed. It will be batch_size*data_dim. Make sure that the
  input is flagged as requires_grad=True with the torch.autograd.Variable
  wrapper. 
  Output: Output of the function for which the Jacobian will
  be computed. It will be batch_size*classes
  Return: Jacobian of dimension batch_size*classes*data_dim
  """
  assert inputs.requires_grad
  print("inside batch diagJ")

  num_classes = output.size()[1] #0 index is batch
  print("num classes", num_classes)

  jacobian = torch.zeros(num_classes, *inputs.size())
  grad_output = torch.zeros(*output.size())
  print(type(jacobian))
  print(device)
  grad_output = grad_output.to(device)
  jacobian = jacobian.to(device)
  print(type(jacobian))

  #zero out gradients
  #compute gradient for one of the classes
  for i in range(num_classes):
    print("i", i)

    zero_gradients(inputs)
    grad_output.zero_()
    grad_output[:,i] = 1
    output.backward(grad_output, retain_graph=True)
    jacobian[i] = inputs.grad.data

  return torch.transpose(jacobian, dim0=0, dim1=1)

time: 13.2 ms


In [7]:

thismodel = 'resnet20_cifar10'
num_classes= 10
num_data = 10000

batch_size = 100

time: 1.82 ms


In [8]:
!pip install pytorchcv
import pytorchcv
from pytorchcv.model_provider import get_model as ptcv_get_model

     |████████████████████████████████| 440kB 3.4MB/s 
time: 3.33 s


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
time: 18.1 s


In [10]:
import sys
sys.path.append("/content/drive/My Drive/Jacobian")
import jacobian as jac
import utils

time: 698 ms


In [11]:
import pytorchcv
from pytorchcv.model_provider import get_model as ptcv_get_model

time: 1.25 ms


In [12]:
train_dataset, test_dataset, train_loader, test_loader = utils.get_data(batch_size=10)


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified
time: 7.19 s


In [13]:
model = ptcv_get_model(thismodel, pretrained=True)  
model.to(device)
model.eval();


time: 819 ms


In [14]:
for left_batch, data in enumerate(test_loader): 
    print(left_batch) 

    inputs, _ = data 
    inputs = inputs.to(device)
    inputs.requires_grad=True
    outputs = model(inputs)
    break

0
time: 95.1 ms


In [15]:
inputs.requires_grad

True

time: 3.29 ms


In [16]:
num_classes = outputs.size()[1] #0 index is batch
print("num classes", num_classes)

num classes 10
time: 954 µs


In [17]:
jacobian = torch.zeros(num_classes, *inputs.size())
grad_output = torch.zeros(*outputs.size())
print(type(jacobian))
print(device)


<class 'torch.Tensor'>
xla:1
time: 2.17 ms


In [18]:
grad_output = grad_output.to(device)
jacobian = jacobian.to(device)
print(type(jacobian))

<class 'torch.Tensor'>
time: 27.3 ms


In [19]:
i = 0
print("i", i)



i 0
time: 1.27 ms


In [20]:
zero_gradients(inputs)


time: 754 µs


In [21]:
grad_output.zero_();


time: 975 µs


In [22]:
grad_output[:,i] = 1;


time: 961 µs


In [23]:
type(grad_output)

torch.Tensor

time: 1.94 ms


In [24]:
type(inputs)

torch.Tensor

time: 1.9 ms


In [29]:
grad_output

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], device='xla:1')

time: 149 ms


In [25]:
#outputs.backward(grad_output, retain_graph=True);


time: 512 µs


In [26]:
#jacobian[i] = inputs.grad.data

time: 573 µs


In [30]:
devices = xm.get_xla_supported_devices(max_devices=8)
devices

['xla:1', 'xla:2', 'xla:3', 'xla:4', 'xla:5', 'xla:6', 'xla:7', 'xla:8']

time: 2.64 ms
